## Image segmentation

Make sure you look at the [Ex_Dealing_with_images.ipynb](Ex_Dealing_with_images.ipynb) notebook first.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## `scikit-image` — Image segmentation

We'll load the luminance data we made before:

In [ ]:
y = np.load("../data/quartz-arenite_LUMINANCE_Michael-C-Rygel_CC-BY-SA.npy")

In [ ]:
plt.imshow(y)
plt.colorbar()

In [ ]:
_ = plt.hist(y.ravel(), bins=128)

In [ ]:
from skimage.filters import try_all_threshold

fig, ax = try_all_threshold(y, figsize=(10, 16), verbose=False)
plt.show()

In [ ]:
from skimage.filters import threshold_otsu

thresh = threshold_otsu(y)
binary = y > thresh

In [ ]:
# A quick aside.
phi = 1 - binary.sum() / binary.size
print(f'porosity = {phi:.1%}')

In [ ]:
plt.imshow(binary)

In [ ]:
from skimage.morphology import closing, opening, square
binary = closing(binary, square(3))
binary = opening(binary, square(3))

In [ ]:
plt.imshow(binary)

In [ ]:
from skimage.morphology import watershed
from skimage.feature import peak_local_max
import scipy.ndimage

distance = scipy.ndimage.distance_transform_edt(binary)
local_maxi = peak_local_max(distance, labels=binary, indices=False, footprint=square(7))
markers, n = scipy.ndimage.label(local_maxi)
labels = watershed(-distance, markers, mask=binary)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15, 5), sharey=True)

axes[0].imshow(binary, cmap='gray', interpolation='nearest')
axes[0].set_title('Overlapping objects')
axes[1].imshow(-distance, cmap='gray', interpolation='nearest')
axes[1].set_title('Distances')
axes[2].imshow(labels, cmap="prism", interpolation='nearest')
axes[2].set_title('Separated objects')

plt.show()

In [ ]:
from skimage.segmentation import mark_boundaries

plt.figure(figsize=(12, 6))
plt.imshow(mark_boundaries(y, labels))

The labeller returned the number of segments: 

In [ ]:
n

<div class="alert alert-success">
<b>Exercise</b>:
<ul>
<li>- Try changing some of the parameters (the thresholding algorithm, the binary operations, the footprint), to see if you can improve the segmentation.</li>
<li>- Find another photograph to work on; see if you can segment it.</li>
</ul>
</div>

## Automatic methods

A quick look at some of `skimage`'s automated segmenters, from [this blog post](http://melvincabatuan.github.io/SLIC-Superpixels/). 

In [ ]:
import skimage.io

img = skimage.io.imread("../data/quartz-arenite_CROPPED_Michael-C-Rygel_CC-BY-SA.png")

In [ ]:
from skimage.segmentation import felzenszwalb, slic, quickshift

segments_fz = felzenszwalb(img, scale=100, sigma=2, min_size=500)
segments_slic = slic(img, n_segments=150, compactness=10, sigma=1)
segments_quick = quickshift(img, kernel_size=5, max_dist=50, ratio=0.25)

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(16, 8, forward=True)
fig.subplots_adjust(0.05, 0.05, 0.95, 0.95, 0.05, 0.05)

ax[0].imshow(mark_boundaries(img, segments_fz))
ax[0].set_title("Felzenszwalbs's method: {} segments".format(len(np.unique(segments_fz))))
ax[1].imshow(mark_boundaries(img, segments_slic))
ax[1].set_title("SLIC: {} segments".format(len(np.unique(segments_slic))))
ax[2].imshow(mark_boundaries(img, segments_quick))
ax[2].set_title("Quickshift: {} segments".format(len(np.unique(segments_quick))))
for a in ax:
    a.set_xticks(())
    a.set_yticks(())
plt.show()
